In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import pandas as pd
from time import sleep
from random import randint
import datetime

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
city = 'Austin'

# Create empty list to store scraped data
datatoget = []

for i in range(0,8):

    # Set url
    url = f'https://austin.craigslist.org/search/austin-tx/apa#search=1~thumb~{i}~0'
    browser.visit(url)
    sleep(randint(5,7))

    # Parse the HTML
    html = browser.html
    soup = bs(html, 'html.parser')

    # Find and extract relevant data
    search = soup.find_all('div', class_='result-info')

    print('\n================')
    print(f'This is page {i+1}')
    print('================')

    # Loop through data found and extract relevant information
    for (counter,link) in enumerate(search):

        # Scrape dates
        try:
            dates = link.find_all('div', class_='meta')[0].text.split('·')[0]

            current_date = datetime.datetime.now()

            if link.find_all('div', class_='meta')[0].text.split('·')[0][-9:] == 'hours ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-11:] == 'minutes ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')
            
            elif link.find_all('div', class_='meta')[0].text.split('·')[0][-8:] == 'hour ago':
                dates = current_date.strftime(f'{current_date.month}/{current_date.day}')

        except:
            dates = np.nan
            

        # Scrape titles    
        titles = link.find_all('a', class_='titlestring')[0].text

        # Scrape links    
        links = link.find_all('a', class_='titlestring', href=True)[0]['href']

        # Scrape prices  
        try:
            prices = link.find_all('span', class_='priceinfo')[0].text
        
        except:
            prices = np.nan

        # Scrape neighborhoods and handle any errors if no neighborhood found   
        try:
            neighborhoods = link.find('div', class_='supertitle').text

        except:
            neighborhoods = np.nan
 
        # Visit each link seperately
        browser.visit(links)
        
        sleep(randint(2,5))

        # After visiting link, extract bath and amenities data
        html = browser.html
        soup = bs(html, 'html.parser')
        
        # Error handeling in case there are no sqft displayed
        try:
            if soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][-3:] == 'ft2':
                sqft = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[2][:-3]
            else:
                sqft = np.nan
        except:
            sqft = np.nan
        
        # Error handeling in case there are no bedrooms displayed
        try:
            bedrooms = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split('/')[0].strip().lower()

        except:
            bedrooms = np.nan
        
        try:
            baths = soup.find_all('p', class_='attrgroup')[0].text.split('\n')[1].split()[2][:-2]

        except:
            baths = np.nan


        try:
            amenities = soup.find_all('p', class_='attrgroup')[1].text.split('\n')
            # List comprehension used to make sure no empty strings get added to the list
            all_amenities = [am for am in amenities if am != '']
        except:
            amenities = np.nan

        # Print statements to ensure that the correct data is scraped
        print(f'\nListing Number: {counter+1}')
        print(f'Square Footage: {sqft if sqft == sqft else "not available"}')
        print(f'Price: {prices if prices == prices else "not available"}')
        print(f'Title: {titles}')
        print(f'Date: {dates}')
        print(f'Number of Bedrooms: {bedrooms if bedrooms == bedrooms else "not available"}')
        print(f'Neighborhood: {neighborhoods if neighborhoods == neighborhoods else "not available"}')
        print(f'Number of Bathrooms: {baths if baths == baths else "not available"}')
        print(f'Link: {links}')
        print(f'Amenities: {all_amenities}')

        # Create dicitonary for our DataFrame
        data = {'date': dates,
                'title': titles,
                'link': links,
                'price': prices,
                'bedroom': bedrooms,
                'sqft': sqft,
                'neighborhood': neighborhoods,
                'bathroom': baths,
                'amenities': all_amenities}

        # Append the data
        datatoget.append(data)

print('\n================ Done ================')

# Quit automated browser instance
browser.quit()


This is page 1

Listing Number: 1
Square Footage: 600
Price: $1,191
Title: Tucked away!! Nice small Community/ Includes cable
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: arboretum/jollyville
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-tucked-away-nice-small-community/7579384568.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d hookups', 'off-street parking', 'rent period: monthly']

Listing Number: 2
Square Footage: 526
Price: $1,425
Title: Gorgeous Area in the Beautiful Hill Country!
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Southwest Austin
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/gorgeous-area-in-the-beautiful-hill/7579384198.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 3
Square Footage: 578
Price: $1,266
Title: Great Views/ 360--
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: we


Listing Number: 20
Square Footage: 540
Price: $1,246
Title: Tucked Away/ Off the beaten Path~ 15 min to Downtown.
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Arboretum
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-tucked-away-off-the-beaten-path/7579382368.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d hookups', 'off-street parking', 'rent period: monthly']

Listing Number: 21
Square Footage: 553
Price: $1,319
Title: UT Shuttle route! Great Neighborhood.
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: far west
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-ut-shuttle-route-great/7579381709.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d hookups', 'off-street parking', 'rent period: monthly']

Listing Number: 22
Square Footage: 750
Price: $1,203
Title: 1Bedroom 1Bath/Northwest Hills/U.T. Shuttle!/Gated!
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Far West
Num


Listing Number: 39
Square Footage: 777
Price: $1,541
Title: (1 MONH FREE!) Luxury Units Just South of Downtown!
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Southpark Meadows
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-monh-free-luxury-units-just/7579373297.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'detached garage', 'rent period: monthly']

Listing Number: 40
Square Footage: 1206
Price: $2,375
Title: Easy Living Near Round Rock
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: North Austin
Number of Bathrooms: 2
Link: https://austin.craigslist.org/apa/d/austin-easy-living-near-round-rock/7579371845.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 41
Square Footage: 1203
Price: $2,369
Title: Apartment Homes in North Austin!
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Round Rock



Listing Number: 58
Square Footage: not available
Price: $590
Title: Looking For A New Start, New Place *******
Date: 1/17
Number of Bedrooms: 6br
Neighborhood: not available
Number of Bathrooms: 4
Link: https://austin.craigslist.org/apa/d/pflugerville-looking-for-new-start-new/7579367612.html
Amenities: ['apartment', 'laundry on site', 'street parking', 'rent period: monthly']

Listing Number: 59
Square Footage: 530
Price: $1,771
Title: ***East ATX Apartment Units***
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: East
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-east-atx-apartment-units/7579365018.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 60
Square Footage: 822
Price: $1,850
Title: ***Luxury in South Austin***
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Southside
Number of Bathrooms: 1
Link: https://austin.craigslist.org/


Listing Number: 78
Square Footage: 639
Price: $1,440
Title: *E. Riverside ATX Apartments*
Date: 1/17
Number of Bedrooms: 1br
Neighborhood: Eastside
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-riverside-atx-apartments/7579362607.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 79
Square Footage: 1079
Price: $1,860
Title: Eastside Stylish Pads!
Date: 1/17
Number of Bedrooms: 2br
Neighborhood: Riverside
Number of Bathrooms: 2
Link: https://austin.craigslist.org/apa/d/austin-eastside-stylish-pads/7579362611.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'off-street parking', 'rent period: monthly']

Listing Number: 80
Square Footage: 534
Price: $1,320
Title: ATX Apartments on East Riverside!
Date: 1/17
Number of Bedrooms: 0br
Neighborhood: East Austin
Number of Bathrooms: 1
L

...


Listing Number: 42
Square Footage: 1570
Price: $3,260
Title: 💸 8 Weeks Free💸 New Downtown Luxury Corner 2 Bed w/ Balcony
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: Downtown East Austin
Number of Bathrooms: 2
Link: https://austin.craigslist.org/apa/d/austin-weeks-free-new-downtown-luxury/7579147260.html
Amenities: ['EV charging', 'air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'no smoking', 'attached garage', 'rent period: monthly', 'wheelchair accessible']

Listing Number: 43
Square Footage: 930
Price: $1,695
Title: 🎊AMAZING ALL NEW!!🎊 ACT NOW!!🎉
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: ROUND ROCK
Number of Bathrooms: 2
Link: https://austin.craigslist.org/apa/d/round-rock-amazing-all-new-act-now/7579156275.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d hookups', 'no smoking', 'street parking', 'rent period: monthly']

Listing Number: 44
Square Footage: 705
Price: $1,125
Title: PRICE JUST LOWE


Listing Number: 61
Square Footage: 1607
Price: $2,310
Title: STOP RENTING - Rent to Own Instead
Date: 1/16
Number of Bedrooms: 3br
Neighborhood: Round Rock
Number of Bathrooms: 2.5
Link: https://austin.craigslist.org/apa/d/round-rock-stop-renting-rent-to-own/7574050690.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'house', 'w/d hookups', 'attached garage', 'rent period: monthly']

Listing Number: 62
Square Footage: 627
Price: $1,195
Title: 💸AWESOME DEAL!! 💸 1 BEDROOM !!!
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: Round Rock
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/round-rock-awesome-deal-bedroom/7579147481.html
Amenities: ['cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d hookups', 'street parking', 'rent period: monthly']

Listing Number: 63
Square Footage: 900
Price: $1,575
Title: 78704 2 bedroom 2 bath, perfect roommate plan
Date: 1/16
Number of Bedrooms: 2br
Neighborhood: South Central
Number of Bathrooms: 2
Link: https:


Listing Number: 80
Square Footage: 2467
Price: $2,699
Title: Brand New 4b/3b House For Rent!
Date: 1/16
Number of Bedrooms: 4br
Neighborhood: not available
Number of Bathrooms: 3
Link: https://austin.craigslist.org/apa/d/austin-brand-new-4b-3b-house-for-rent/7576999693.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d hookups', 'no smoking', 'attached garage', 'rent period: monthly']

Listing Number: 81
Square Footage: not available
Price: $1,654
Title: Catch Every Austin FC Game! | BAD CREDIT OK
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: Arboretum
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-catch-every-austin-fc-game-bad/7579144104.html
Amenities: ['air conditioning', 'broker fee details: N/A', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'listed by: WTA Realty', 'off-street parking', 'rent period: monthly']

Listing Number: 82
Square Footage: 1236
Price: $1,775
Title: Ga


Listing Number: 98
Square Footage: 720
Price: $1,427
Title: Spacious and beautiful. Consider your new 1 bed / 1 bath found!
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: Austin
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-spacious-and-beautiful-consider/7579135760.html
Amenities: ['application fee details: $55', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'laundry on site', 'off-street parking', 'rent period: monthly']

Listing Number: 99
Square Footage: 351
Price: $1,144
Title: Pet Friendly (Pet Fees Apply), New Light Fixtures, Karaoke Room
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: 8300 North Interstate Highway 35, Austin, TX
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-pet-friendly-pet-fees-apply-new/7577709337.html
Amenities: ['apartment', 'laundry on site', 'off-street parking', 'rent period: monthly']

Listing Number: 100
Square Footage: 650
Price: $1,452
Title: Great Pet-Friendly Community in Austin! 


Listing Number: 116
Square Footage: 615
Price: $1,532
Title: ** cozy 1/1 -- QUICK MOVE-IN-- BAD CREDIT-OK! **
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: Northwest
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/austin-cozy-quick-move-in-bad-credit-ok/7576660361.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d in unit', 'street parking', 'rent period: monthly']

Listing Number: 117
Square Footage: 627
Price: $1,528
Title: MODERN -- great layout -- BAD CREDIT - OK!
Date: 1/16
Number of Bedrooms: 1br
Neighborhood: Pflugerville
Number of Bathrooms: 1
Link: https://austin.craigslist.org/apa/d/pflugerville-modern-great-layout-bad/7570507196.html
Amenities: ['air conditioning', 'cats are OK - purrr', 'dogs are OK - wooof', 'apartment', 'w/d hookups', 'street parking', 'rent period: monthly']

Listing Number: 118
Square Footage: not available
Price: $1,931
Title: NEED TO MOVE THIS MONTH? -- let's make it happen!
Date: 

In [6]:
# creating DataFrame
df = pd.DataFrame(datatoget)
df

,date,title,link,price,bedroom,sqft,neighborhood,bathroom,amenities
0,1/17,Tucked away!! Nice small Community/ Includes c...,https://austin.craigslist.org/apa/d/austin-tuc...,"$1,191",1br,600,arboretum/jollyville,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
1,1/17,Gorgeous Area in the Beautiful Hill Country!,https://austin.craigslist.org/apa/d/gorgeous-a...,"$1,425",1br,526,Southwest Austin,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
2,1/17,Great Views/ 360--,https://austin.craigslist.org/apa/d/austin-gre...,"$1,266",1br,578,west/ 360,1,"[cats are OK - purrr, dogs are OK - wooof, apa..."
3,1/17,"Newly Painted, Pet Friendly (Pet Fees Apply), ...",https://austin.craigslist.org/apa/d/austin-new...,"$1,144",1br,351,"8300 North Interstate Highway 35, Austin, TX",1,"[air conditioning, cats are OK - purrr, dogs a..."
4,1/17,"Vinyl Plank Flooring, Pass-Through Serving Bar...",https://austin.craigslist.org/apa/d/round-rock...,"$1,775",4br,1236,"2500 Louis Henna Boulevard, Round Rock, TX",2,"[air conditioning, cats are OK - purrr, dogs a..."
...,...,...,...,...,...,...,...,...,...
955,1/16,** cozy 1/1 -- QUICK MOVE-IN-- BAD CREDIT-OK! **,https://austin.craigslist.org/apa/d/austin-coz...,"$1,532",1br,615,Northwest,1,"[air conditioning, cats are OK - purrr, dogs a..."
956,1/16,MODERN -- great layout -- BAD CREDIT - OK!,https://austin.craigslist.org/apa/d/pflugervil...,"$1,528",1br,627,Pflugerville,1,"[air conditioning, cats are OK - purrr, dogs a..."
957,1/16,NEED TO MOVE THIS MONTH? -- let's make it happen!,https://austin.craigslist.org/apa/d/austin-nee...,"$1,931",2br,NaN,Northeast (Parmer ln/ I-35),1,"[air conditioning, cats are OK - purrr, dogs a..."
958,1/16,💸 4 Weeks Free💸 Brand New Luxury 1 Bed 15 Min ...,https://austin.craigslist.org/apa/d/austin-wee...,"$1,565",1br,730,East Riverside Riverside Pleasant Valley,1,"[EV charging, air conditioning, cats are OK - ..."


In [9]:
# Exporting data to csv
df.to_csv(f'{city}_data_raw.csv', index=False)

In [10]:
# Quit automated browser instance
browser.quit()